In [ ]:
# @title import
!pip install pulp
import pulp as pl

In [ ]:
# @title Данные
q = [70, 60, 50]  # цена за баррель бензина
p = [45, 35, 25]  # цена за баррель нефти
d = [3000, 10000, 500]  # потребность в бензине
c = [5000, 5000, 5000]  # закупленное количество баррелей нефти

O = [12, 6, 8]  # октановое число в сырой нефти
o = [10, 8, 6]  # октановое число в бензине
S = [0.005, 0.02, 0.03]  # количество серы в сырой нефти
s = [0.01, 0.02, 0.01]  # количество серы в бензине
C = 14000 # мощность предприятия
b = 4  # стоимость производства барреля бензина
delta = 10  # прирост потребности в бензине на каждый доллар, вложенный в рекламу

In [ ]:
# @title Переменные

problem = pl.LpProblem("Gasoline_production", pl.LpMaximize)

# Переменные решения
x = [[pl.LpVariable(f"x{i}{j}", lowBound=0) for j in range(len(o))] for i in range(len(c))]
g = [pl.LpVariable(f"g{j}", lowBound=0) for j in range(len(o))]
a = [pl.LpVariable(f"a{j}", lowBound=0) for j in range(len(o))]

In [ ]:
# @title Целевая функция
objective = -pl.lpSum([p[i] * c[i] for i in range(len(c))]) + pl.lpSum([q[j] * g[j] for j in range(len(d))]) - pl.lpSum([a[j] for j in range(len(d))]) - b * pl.lpSum([g[j] for j in range(len(d))])
problem += objective

In [ ]:
# @title Ограничения
for i in range(len(c)):
    problem += pl.lpSum([x[i][j] for j in range(len(o))]) <= c[i]

for j in range(len(o)):
    problem += pl.lpSum([x[i][j] for i in range(len(c))]) == g[j]
    problem += g[j] == d[j] + delta * a[j]
    problem += pl.lpSum([S[i] * x[i][j] for i in range(len(c))]) <= s[j] * pl.lpSum([x[i][j] for i in range(len(c))])
    problem += pl.lpSum([O[i] * x[i][j] for i in range(len(c))]) >= o[j] * pl.lpSum([x[i][j] for i in range(len(c))])

problem += pl.lpSum(g) <= C

In [ ]:
# @title Решение

problem.solve()

print("Оптимальное решение:")
for i in range(len(c)):
  print(f'Бензин типа {i+1}:')
  for j in range(len(o)):
      print(f"\tНефть типа {j}:", pl.value(x[j][i]))
  print(f"\tКолличество баррелей {i+1} типа бензина:", pl.value(g[i]))
  print(f"\tКолличество денег потраченных на рекламу {i+1} типа бензина:", pl.value(a[i]), end="\n\n")
print("Прибыль:", pl.value(problem.objective))